# 유사한 웹툰 가져오기

## 중복구현

In [1]:
import pandas as pd
import numpy as np
import os

path = '../Web Service/data/'
web = pd.read_csv(os.path.join(path, 'all.csv'), encoding='cp949')

In [2]:
web.head()

,title,writer,illustrator,genre,story,platform,thumb,url
0,나라 구한 전생,양키풀,양키풀,드라마,아버지의 빚 문제로 폐공장에 납치되어 있던 한자인은 재수 좋은 남자 천운조의 등장으...,네이버 웹툰,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...
1,모죠의 일지,모죠,모죠,개그,소신있는 취향과 내성적인 성향.훌륭한 집순이 모죠의 은밀한 비밀들,네이버 웹툰,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...
2,밤하늘에 구름운,견지,그리온,로맨스,"망겜 <검풍 온라인>의 NPC 초낙운, 서윤의 평범했던 일상에 등장!?NPC와의 좌...",네이버 웹툰,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...
3,롤랑롤랑,자유,자유,판타지,코기 왕자 롤랑과 마녀 루이사의 모험 이야기.베일에 숨겨져 있던 대사제의 과거와 교...,네이버 웹툰,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...
4,샹치,진 루엔 양,다이크 루언,액션,지도자 젱 주의 죽음 이후 모습을 드러내지 않았던 사악한 고대 비밀 집단.하지만 힘...,네이버 웹툰,https://shared-comic.pstatic.net/thumb/webtoon...,https://comic.naver.com/webtoon/list?titleId=7...


In [3]:
# 매트릭스 불러오기
matrices_path = path + 'matrices/'

okt_matrices = np.load(matrices_path + 'okt.npy', allow_pickle=True)
kkma_matrices = np.load(matrices_path + 'kkma.npy', allow_pickle=True)
hannanum_matrices = np.load(matrices_path + 'hannanum.npy', allow_pickle=True)
komoran_matrices = np.load(matrices_path + 'komoran.npy', allow_pickle=True)

## 행렬 합치기
- 작가 요소 추가/제거, 작가와 장르 가중치 설정

In [4]:
from scipy.sparse import hstack

def hstack_matrices(matrices, noauthor, w_author, w_genre):
    cnt_genre_matrix, lb_writer_matrix, lb_illustrator_matrix, tfidf_story_matrix = matrices
    
    # tfidf_title_matrix,
    # 만들어진 행렬을 합치기 위한 리스트
    sparse_matrix_list = [cnt_genre_matrix * w_genre, tfidf_story_matrix]
    if not noauthor:
        sparse_matrix_list.append(lb_writer_matrix * w_author)
        sparse_matrix_list.append(lb_illustrator_matrix * w_author)
    
    sparse_matrix_list = tuple(sparse_matrix_list)

    # 행렬 결합
    X_features_sparse = hstack(sparse_matrix_list).tocsr()
#     print(type(X_features_sparse), X_features_sparse.shape)
    return X_features_sparse

In [5]:
# 웹툰 제목을 입력하면 해당 인덱스를 반환해줄 Series 생성
indices = pd.Series(web.index, index=web['title'])
indices.head(3)

title
나라 구한 전생    0
모죠의 일지      1
밤하늘에 구름운    2
dtype: int64

## 유사한 스토리/장르 등을 가진 웹툰 추천

In [6]:
from itertools import groupby
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 합칠 행렬
tokenizer_matrices = [okt_matrices, kkma_matrices, hannanum_matrices, komoran_matrices]

def get_recomm(*title, rank=10, same_genre=False, noauthor=False,  w_author=0.7, w_genre=1.3, tokenizer_matrices=tokenizer_matrices):
    # 코사인 유사도를 담을 array 생성
    cosine_sim = np.zeros((len(web), len(web)))
    
    # 각 토큰화 도구로 만들어진 행렬을 합쳐서 코사인 유사도 생성
    for matrices in tokenizer_matrices:
        X_features_sparse = hstack_matrices(matrices, noauthor=noauthor,  w_author=w_author, w_genre=w_genre)
        cosine_sim += cosine_similarity(X_features_sparse, X_features_sparse)
    cosine_sim /= len(tokenizer_matrices)
    
    # 결과 출력을 위한 데이터 프레임 복사
    print_copy = web.copy()

    # 데이터프레임 구성
    t_df = pd.DataFrame(columns=['title', 'writer', 'illustrator', 'genre', 'story', 'platform', 'similarity'])
    
    # 다중검색
    for t in title:
        # 선택한 웹툰 제목으로부터 해당되는 인덱스를 받아옴
        idx = indices[t]
        if idx.size > 1:
            print(print_copy.iloc[idx,[1,2,4]])
            temp = int(input('원하는 작품을 선택하세요. : '))
            idx = idx[temp-1]
        else:
            pass

        # 인덱스로부터 장르를 뽑아옴
        genre = print_copy.loc[idx, 'genre']

        # 유사도를 DataFrame에 붙임
        print_copy['similarity'] = cosine_sim[idx]
    
        if same_genre:
            # 동일 장르만을 보여줄 경우
            df =  print_copy[print_copy['genre'] == genre].sort_values(by='similarity', ascending=False)
        
        else:
            # 다른 장르도 보여줄 경우
            df = print_copy.sort_values(by='similarity', ascending=False)
        t_df = pd.concat([t_df,df])
    
    # 작품끼리 유사도 더하기
    t_df = t_df.groupby(['title','writer','illustrator', 'genre','story','platform'],as_index=False).sum()
    
    
    return t_df.sort_values(by='similarity',ascending=False).iloc[len(title):rank+len(title)+1,:]

### 중복처리

In [7]:
idx = indices['트리거']
type(idx)

pandas.core.series.Series

In [8]:
idx.size

2

In [9]:
idx

title
트리거    1112
트리거    1872
dtype: int64

In [10]:
web.iloc[1112]

title                                                        트리거
writer                                                       고경빈
illustrator                                                  고경빈
genre                                                        판타지
story          지구에 소행성이 떨어진 후, 지구는 인간을 먹이로 하는 괴생명체 '아니스'에 의해 ...
platform                                                  네이버 웹툰
thumb          https://shared-comic.pstatic.net/thumb/webtoon...
url            https://comic.naver.com/webtoon/list?titleId=7...
Name: 1112, dtype: object

In [11]:
%%time
get_recomm('신의 탑')

Wall time: 835 ms


,title,writer,illustrator,genre,story,platform,similarity
531,나 혼자 만렙 뉴비,"메슬로우,WAN.Z",스윙뱃,판타지,"게임 너튜버로 활동 중인 주인공 진혁은 유일하게 [시련의 탑]의 엔딩을 봤지만, ...",네이버 웹툰,0.482053
2975,창위의일루젼,시우,시우,판타지,눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.473349
3508,환상적인 소년,후은,후은,판타지,평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.472054
1544,빌런투킬,퓨핀,은지,판타지,동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.471984
105,EVE,은유,콩자,판타지,극대화된 오감을 시험 봐서 등수대로 등에 번호가 매겨지는 'EVE'.이곳에서 1번부...,네이버 웹툰,0.471714
1720,세상에 없는 이웃,1230,1230,판타지,"구원하소서 , 방황하는 신 으로 밀도 높은 비주얼과 독특한 시나리오로 독자를 매료시...",레진코믹스,0.471667
304,골든 체인지,브림스,브림스,판타지,세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.471451
2373,오로지 오로라,홍달,홍달,판타지,"마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 ...",네이버 웹툰,0.470616
1485,붉은 실,동비,동비,판타지,생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.469842
2987,천년구미호,기량,기량,판타지,세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.469355


In [12]:
get_recomm('트리거')

     writer illustrator                                              story
1112    고경빈         고경빈  지구에 소행성이 떨어진 후, 지구는 인간을 먹이로 하는 괴생명체 '아니스'에 의해 ...
1872   불만있냥        불만있냥  불량하기로 유명한 대명 고등학교에 입학한 평범한 고등학생 남지일.\n입학 첫날부터 ...
원하는 작품을 선택하세요. : 1


,title,writer,illustrator,genre,story,platform,similarity
1089,마물전담부대,염손,염손,판타지,마법혁명으로 근대화를 이룬 판타지 세계.하지만 이러한 세계에서도 마물은 여전히 인류...,네이버 웹툰,0.472252
1188,멸망X초이스,텐,텐,판타지,"수능 날의 간절한 마음이 통한 건지, 정답을 알려주는 능력을 가지게 된 서준. 이 ...",네이버 웹툰,0.471473
3385,하이브 1~2,김규삼,김규삼,판타지,거대해진 곤충들이 인간을 습격한다! 원시시대로 돌아간 지구의 대기상태,네이버 웹툰,0.471160
3481,혼모노트,민수상,민수상,판타지,"평온했던(?) 교실 안에 갑자기 트럭이 날아 오더니, 순식간에 반 아이들은 이세계...",네이버 웹툰,0.470724
3383,하우스키퍼,채용택,유현,판타지,'A.I가 인간에게 반역하는 것은 인간을 위해서이다.' A.I 혁명이 일어난 세계에...,네이버 웹툰,0.470228
3070,카루나,강호진,강호진,판타지,지옥의 군대와 맞서는 항마승병 카루나. 카루나에 도전하는 소년액션 모험 판타지!,네이버 웹툰,0.469958
1988,아르마,병장,병장,판타지,"인간을 해칠 수 없는 초능력자, 인류의 구원이 되다!",네이버 웹툰,0.469910
31,2012 지구가 멸망한다면?,웹툰작가,웹툰작가,판타지,"2011년을 뜨겁게 달군 '미스테리 단편'을 잇는 2012년 특집 웹툰, <2012...",네이버 웹툰,0.469762
3074,카오스어택,맛스타,맛스타,판타지,지구는 바야흐로 영웅시대평범한 고등학생과 엉뚱발랄 영웅들의 좌충우돌 에피소드!,네이버 웹툰,0.469587
1544,빌런투킬,퓨핀,은지,판타지,동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.468710


In [13]:
get_recomm('트리거')

     writer illustrator                                              story
1112    고경빈         고경빈  지구에 소행성이 떨어진 후, 지구는 인간을 먹이로 하는 괴생명체 '아니스'에 의해 ...
1872   불만있냥        불만있냥  불량하기로 유명한 대명 고등학교에 입학한 평범한 고등학생 남지일.\n입학 첫날부터 ...
원하는 작품을 선택하세요. : 2


,title,writer,illustrator,genre,story,platform,similarity
279,고교호구왕,김구일,슥수,"액션, 무협","살아있는 악마라 불리는 일진, 경태의 셔틀 선한결! \n중학교 3년 내내 한결이라...",카카오페이지,0.640992
1073,마검왕,나민채,소구미,"액션, 무협",평범한 고등학생 진욱의 이중생활\n\n무림과 현실을 넘나드는 액션 판타지,카카오페이지,0.639686
1955,싸움의 기술2,신한솔,구한준,"액션, 무협",2000년대 후반을 배경으로 한 왕따 고교생이 전설적인 싸움꾼을 만나 진정한 남자로...,카카오페이지,0.639240
1236,무령검존,옥타브,옥타브,"액션, 무협","최강의 무령검존(武 尊)으로 불리었던 사내,\n갑작스러운 죽음 뒤 약골의 몸으로 ...",카카오페이지,0.639135
198,강신전기,증기 스튜디오x텐센트 동만,증기 스튜디오x텐센트 동만,"액션, 무협",*세계에는 이계에 사는 권신이라고\n불리는 종족들을 소환하여 일생을 함께하며\n수련...,카카오페이지,0.638085
1477,불사신왕무쌍기,two cats,two cats,"액션, 무협",심장병으로 몸이 약했던 진현. 어느 날 우연히 고양이로 인해 '불사신석'의 주인이 ...,카카오페이지,0.638075
1127,만계선종,아이리더,아이리더,"액션, 무협",16년 전 성운은 약한 몸을 가지고 태어난다.\n\n성운의 조부는 약한 아이가 걱정...,카카오페이지,0.637750
1254,무인이곽,우각,류금철,"액션, 무협","평범한 무인이었던 이곽, 신마련의 관일현에게 무참히 폭행을 당하게 되고, 사지를 쓸...",카카오페이지,0.637476
277,고교정점,스튜디오 문,스튜디오 문,"액션, 무협","거침없이 살던 그 시절 나는 무서울 것이 없었고,\n어느새 나에게는 고교정점이라는 ...",카카오페이지,0.636808
273,고교권왕,김성모,김성모,"액션, 무협",국내 굴지그룹인 바이오유업의 장자인 한류. \n계모의 위협으로부터 자신의 몸을 지키...,카카오페이지,0.636608


In [14]:
get_recomm('이태원 클라쓰', w_author=1.5, w_genre=1.0)

,title,writer,illustrator,genre,story,platform,similarity
1072,링크보이,광진,두엽,판타지,약육강식의 무법도시.잘못된 이 도시를 바로잡는다.,네이버 웹툰,0.346859
1628,사장님은 곰인중,유승종,유승종,드라마만화,사장님의 성추행에 항의하다 부당하게 해고당한 여담! \n재취직을 위해 고군분투하지만...,카카오페이지,0.158104
2328,영혼의 요리사,도토리묵,김렉나,드라마만화,"낮에는 사람들에게 음식을 팔고, \n밤에는 죽은 영혼들에게 음식을 대접하는 푸드트럭...",카카오페이지,0.157071
3305,퐁당퐁당,위성우,위성우,드라마만화,[스크롤 방식으로 최적화되어 있습니다.]\n\n대기업의 잘나가는 엘리트 기획팀장 강...,카카오페이지,0.156989
2766,저녁 같이 드실래요?,박시인,박시인,드라마만화,나에게도 꽃미남 밥 친구를! 음식에 얽힌 두 남녀의 비밀스런 연애 이야기,카카오페이지,0.156837
2243,여긴 불빛이 안 드는 곳이야,완두콩,완두콩,드라마만화,사람은 누구나 자신만의 고민을 가지고 있다.\n\n고등학교 1학년 청춘을 맞이한 여...,카카오페이지,0.156413
2322,영원의 안식처 (Eternal Sabbath),Fuyumi Souryo,Fuyumi Souryo,드라마만화,"그(ES)는 타인의 뇌(마음) 속으로 파고들어, 슬쩍 기억을 바꿔놓는다.\n""ES""...",카카오페이지,0.156322
1300,미생1,윤태호,윤태호,드라마만화,자신의 삶을 승리하기 위해 한 수 한 수 돌을 잇는 사람들의 이야기,카카오페이지,0.156320
720,논현동 장사꾼,"김영현, 아론",구김,드라마만화,젊은 패기로 시작했던 장사를 4번이나 말아먹고 빚더미에 나앉게 된 27세 청년 '성...,카카오페이지,0.156275
508,꽃미남 게스트하우스 IN 북촌,송지은,김병관,드라마만화,사진을 찍는 족족 화보가 되어버리는 곳! '북촌 8경' 이 정해져 있을 정도로 골목...,카카오페이지,0.156227


In [15]:
get_recomm('이태원 클라쓰', w_author=1.0, w_genre=1.5)

,title,writer,illustrator,genre,story,platform,similarity
1628,사장님은 곰인중,유승종,유승종,드라마만화,사장님의 성추행에 항의하다 부당하게 해고당한 여담! \n재취직을 위해 고군분투하지만...,카카오페이지,0.433843
2328,영혼의 요리사,도토리묵,김렉나,드라마만화,"낮에는 사람들에게 음식을 팔고, \n밤에는 죽은 영혼들에게 음식을 대접하는 푸드트럭...",카카오페이지,0.432564
3305,퐁당퐁당,위성우,위성우,드라마만화,[스크롤 방식으로 최적화되어 있습니다.]\n\n대기업의 잘나가는 엘리트 기획팀장 강...,카카오페이지,0.432462
2766,저녁 같이 드실래요?,박시인,박시인,드라마만화,나에게도 꽃미남 밥 친구를! 음식에 얽힌 두 남녀의 비밀스런 연애 이야기,카카오페이지,0.432275
2243,여긴 불빛이 안 드는 곳이야,완두콩,완두콩,드라마만화,사람은 누구나 자신만의 고민을 가지고 있다.\n\n고등학교 1학년 청춘을 맞이한 여...,카카오페이지,0.431749
2322,영원의 안식처 (Eternal Sabbath),Fuyumi Souryo,Fuyumi Souryo,드라마만화,"그(ES)는 타인의 뇌(마음) 속으로 파고들어, 슬쩍 기억을 바꿔놓는다.\n""ES""...",카카오페이지,0.431637
1300,미생1,윤태호,윤태호,드라마만화,자신의 삶을 승리하기 위해 한 수 한 수 돌을 잇는 사람들의 이야기,카카오페이지,0.431634
720,논현동 장사꾼,"김영현, 아론",구김,드라마만화,젊은 패기로 시작했던 장사를 4번이나 말아먹고 빚더미에 나앉게 된 27세 청년 '성...,카카오페이지,0.431578
508,꽃미남 게스트하우스 IN 북촌,송지은,김병관,드라마만화,사진을 찍는 족족 화보가 되어버리는 곳! '북촌 8경' 이 정해져 있을 정도로 골목...,카카오페이지,0.431520
2221,에이스튜던트,사이,강갱,드라마만화,"""선생님! 저를 다시 1등으로 만들어 주세요!"" \n\n어려서부터 자신을 괴롭혀왔던...",카카오페이지,0.431431


### 다중 검색

In [16]:
get_recomm('왕으로 살다', '비흔', w_author=1.0, w_genre=1.5)

,title,writer,illustrator,genre,story,platform,similarity
3175,태왕광개토,청안랑,김인호,시대극,고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.865850
3410,한섬세대,유승진,유승진,시대극,"조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연...",네이버 웹툰,0.865243
2985,천국의 신화 6부,올댓스토리,이현세,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.864950
2021,"아이소포스 1, 2부",김양수,도가도,시대극,"이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.862919
1643,삵의 발톱,손영완,손영완,시대극,"중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바...",네이버 웹툰,0.862500
1648,삼국지톡,무적핑크,이리,시대극,"""유비, 조조, 손권 등 영웅들이 활약하던 후한, 그 난세에 핸드폰이 있었다면?메신...",네이버 웹툰,0.861472
2022,아이소포스 3부,김양수,도가도,시대극,"이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.아이소포스가 3부로 돌아왔다",네이버 웹툰,0.861443
3590,흐드러지다,연제원,연제원,시대극,"지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.861049
1279,문아,팬마,팬마,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.860089
2853,조선방랑야사,가현,가현,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.859954


In [17]:
get_recomm('삼국지톡', '삵의 발톱', w_author=1.0, w_genre=1.5)

,title,writer,illustrator,genre,story,platform,similarity
1536,비흔,정재한,황영찬,시대극,"잊혀진 나라 백제, 그리고 싸울아비.패자의 역사 뒤로 전설이 되어 사라진 검은 이리...",네이버 웹툰,0.865537
3175,태왕광개토,청안랑,김인호,시대극,고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.861161
2985,천국의 신화 6부,올댓스토리,이현세,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.861126
1279,문아,팬마,팬마,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.859359
2853,조선방랑야사,가현,가현,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.859282
2430,왕으로 살다,네스티캣,네스티캣,시대극,살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.858435
2385,오성X한음,유승진,유승진,시대극,"영길리국에 셜록과 왓슨이 있다면 조선에는 이들이 있다!조선판 수사추리극, 오성X한음...",네이버 웹툰,0.858271
3590,흐드러지다,연제원,연제원,시대극,"지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.858150
3496,화산귀환,비가,LICO,시대극,대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일...,네이버 웹툰,0.858118
1228,몽홀,장태산,장태산,시대극,차갑고 척박한 땅 몽홀 그 곳에서 펼쳐지는 감동의 대서사시,네이버 웹툰,0.858112
